<a href="https://colab.research.google.com/github/MateusFreitas-C/Branch_and_Bound/blob/main/Branch_and_bound.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mip
from mip import *
import math as mt

In [ ]:
# Função para ler os dados das instancias

def load_data_inst(file_path):
  with open(file_path, 'r') as file:
    first_line = file.readline().strip()
    vars_qty, constr_qty = map(int, first_line.split())

    second_line = file.readline().strip()
    k = list(map(float, second_line.split()))

    c = []
    for _ in range(constr_qty):
        line = file.readline().strip()
        c.append(list(map(float, line.split())))

    return vars_qty, constr_qty, k, c

In [ ]:
def save(model, filename):
  model.write(filename)
  with open(filename, 'r') as file:
      return file.read()

In [ ]:
stack = []

In [ ]:
def branch_and_bound(model, x, fp):
    global zp, zd
    global iter

    # Criando cópias do modelo para o ramo inferior e superior
    model_lower = model.copy()
    model_upper = model.copy()

    # Otimiza o modelo atual
    status = model.optimize()

    fp.write("--" * 70 + "\n")
    fp.write(f"Iteração {iter}\n" +
           (f"Resultado da Relaxação Linear: {model.objective_value:.3f}\n" if status != OptimizationStatus.INFEASIBLE else "\n"))

    # Adiciona o índice atual à pilha de iterações e incrementa iterador
    stack.insert(0, iter)
    iter += 1

    # Verificação de poda por inviabilidade
    if status in (OptimizationStatus.INFEASIBLE, OptimizationStatus.NO_SOLUTION_FOUND):
        return None, False

    # Atualiza o limite dual (zd) se o valor do objetivo atual for menor
    zd = min(zd, model.objective_value)

    fp.write("Variáveis: \n")
    for v in model.vars:
      fp.write(f"{v.name} = {v.x:.2f}\n")

    # Inicialização de variáveis para verificar se há solução fracionária
    best_var = None
    found_fractional = False
    closest_to_half = 0.5  # Define uma base inicial para verificar quão próxima uma variável está de 0.5

    # Itera sobre as variáveis do modelo para encontrar a mais próxima de 0.5
    for v in model.vars:
        if 0 < v.x < 1:  # Verifica se a variável é fracionária
            dist_to_half = abs(v.x - 0.5)
            if dist_to_half < closest_to_half:
                closest_to_half = dist_to_half
                best_var = v
                found_fractional = True

    # Se não houver variáveis fracionárias, a solução é inteira, podemos retornar o modelo
    if not found_fractional:
        return model, True

    fp.write(f"Variável escolhida: {best_var} = {best_var.x:.2f}\n")

    # Poda por limitante (verifica se a solução atual é pior que a melhor já encontrada)
    if zp >= model.objective_value:
        return model, False

    fp.write(f"\nRestrições:")
    constraints = save(model, "model.lp")
    fp.write(f"\n\n{constraints}\n")

    # Ramo esquerdo: adiciona a restrição de que a melhor variável fracionária seja 0
    model_lower += best_var == 0
    left_result, is_integral = branch_and_bound(model_lower, x, fp)

    # Remove a iteração da pilha após a chamada recursiva
    stack.pop(0)
    if is_integral and (zp == -mt.inf or left_result.objective_value > zp):
        zp = left_result.objective_value

    # Ramo direito: adiciona a restrição de que a melhor variável fracionária seja 1
    model_upper += best_var == 1
    right_result, is_integral = branch_and_bound(model_upper, x, fp)

    # Remove a iteração da pilha após a chamada recursiva
    stack.pop(0)
    if is_integral and (zp == -mt.inf or right_result.objective_value > zp):
        zp = right_result.objective_value

    # Retorna o melhor resultado encontrado ou o modelo atual se nenhum foi encontrado
    return left_result if left_result is not None else model, False


In [ ]:
def run_tests():
  for i in range(4):
    global zp, zd
    global iter

    zp, zd = -mt.inf, -mt.inf
    iter = 1

    vars_qty, constr_qty, k, c = load_data_inst(f"teste{i+1}.txt")

    model = Model(sense=MAXIMIZE, solver_name=CBC)

    x = [model.add_var(var_type=CONTINUOUS, name=f"x_{j}", lb=0, ub=1) for j in range(vars_qty)]
    model.objective = xsum(k[j] * x[j] for j in range(vars_qty))

    for n in range(constr_qty):
        model += xsum(c[n][j] * x[j] for j in range(vars_qty)) <= c[n][len(c[n]) - 1]

    fres = open(f"resposta{i+1}.txt", "w")

    branch_and_bound(model, x, fres)

    print(f"Solution Value {i+1}: {zp}")
    fres.write(f"Solution Value {i+1}: {zp}")

    fres.close()

run_tests()

Solution Value 1: 20.0
Solution Value 2: 24.0
Solution Value 3: 19.0
Solution Value 4: 10.0


In [ ]:
fres = open(f"resposta1.txt", "r")
print(fres.read())
fres.close()

--------------------------------------------------------------------------------------------------------------------------------------------
Iteração 1
Resultado da Relaxação Linear: 28.750
Variáveis: 
x_0 = 0.00
x_1 = 0.87
x_2 = 0.00
x_3 = 0.00
x_4 = 1.00
x_5 = 1.00
x_6 = 0.00
Variável escolhida: x_1 = 0.87

Restrições:

\Problem name: 

Minimize
OBJROW: -2 x_0 -10 x_1 -8 x_2 -7 x_3 -10 x_4 -10 x_5 -6 x_6
Subject To
constr(0):  5 x_0 + 7 x_1 + 8 x_2 + x_3 + 7 x_4 + 5 x_5 + 6 x_6 <= 20
constr(1):  x_0 + 6 x_1 + 4 x_2 + 9 x_3 + 10 x_4 + 6 x_5 + 10 x_6 <= 30
constr(2):  4 x_0 + 4 x_1 + 4 x_2 + x_3 + 5 x_4 + 5 x_5 + 10 x_6 <= 40
constr(3):  3 x_0 + 10 x_1 + 8 x_2 + x_3 + 3 x_4 + 3 x_5 + 8 x_6 <= 30
constr(4):  10 x_0 + 8 x_1 + 9 x_2 + 9 x_3 + 7 x_4 + 6 x_5 + 10 x_6 <= 20
constr(5):  6 x_0 + 6 x_1 + 3 x_2 + 6 x_3 + 3 x_4 + 7 x_5 + 2 x_6 <= 80
constr(6):  7 x_0 + 10 x_1 + 7 x_2 + 8 x_3 + 7 x_4 + 8 x_5 + 7 x_6 <= 100
constr(7):  9 x_0 + 8 x_1 + x_2 + x_3 + 8 x_4 + 10 x_5 + 2 x_6 <= 90
constr

In [ ]:
fres = open(f"resposta2.txt", "r")
print(fres.read())
fres.close()

--------------------------------------------------------------------------------------------------------------------------------------------
Iteração 1
Resultado da Relaxação Linear: 28.455
Variáveis: 
x_0 = 0.00
x_1 = 1.00
x_2 = 0.09
x_3 = 0.00
x_4 = 0.00
x_5 = 1.00
x_6 = 1.00
x_7 = 0.00
x_8 = 0.55
Variável escolhida: x_8 = 0.55

Restrições:

\Problem name: 

Minimize
OBJROW: -7 x_0 -7 x_1 -7 x_2 -5 x_3 -8 x_4 -8 x_5 -9 x_6 -10 x_7 -7 x_8
Subject To
constr(0):  x_0 + 3 x_1 + x_2 + 3 x_3 + 3 x_4 + 7 x_5 + 2 x_6 + x_7 + 4 x_8 <= 80
constr(1):  7 x_0 + 6 x_1 + 10 x_2 + x_3 + 7 x_4 + 2 x_5 + 2 x_6 + 7 x_7 + x_8 <= 90
constr(2):  3 x_0 + 2 x_1 + x_2 + 3 x_3 + 3 x_4 + 2 x_5 + x_6 + 6 x_7 + 5 x_8 <= 10
constr(3):  10 x_0 + 8 x_1 + 3 x_2 + 6 x_3 + 10 x_4 + 7 x_5 + 3 x_6 + 10 x_7 + 4 x_8 <= 30
constr(4):  2 x_0 + 8 x_1 + 6 x_2 + 5 x_3 + 6 x_4 + 6 x_5 + 9 x_6 + 7 x_7 + 2 x_8 <= 80
constr(5):  3 x_0 + 10 x_1 + x_2 + 9 x_3 + 2 x_4 + 7 x_5 + 7 x_6 + 9 x_7 + 10 x_8 <= 90
constr(6):  x_0 + 7 x_1 + 1

In [ ]:
fres = open(f"resposta3.txt", "r")
print(fres.read())
fres.close()

--------------------------------------------------------------------------------------------------------------------------------------------
Iteração 1
Resultado da Relaxação Linear: 24.913
Variáveis: 
x_0 = 0.00
x_1 = 0.00
x_2 = 1.00
x_3 = 0.00
x_4 = 0.00
x_5 = 0.00
x_6 = 1.00
x_7 = 0.13
x_8 = 0.61
Variável escolhida: x_8 = 0.61

Restrições:

\Problem name: 

Minimize
OBJROW: -7 x_0 -9 x_1 -10 x_2 -3 x_3 -6 x_4 - x_5 -9 x_6 -8 x_7 -8 x_8
Subject To
constr(0):  2 x_0 + x_1 + 9 x_2 + 6 x_3 + 3 x_4 + 6 x_5 + 10 x_6 + 9 x_7 + x_8 <= 60
constr(1):  8 x_0 + 6 x_1 + 6 x_2 + 5 x_3 + 2 x_4 + 2 x_5 + 4 x_6 + 3 x_7 + 6 x_8 <= 80
constr(2):  8 x_0 + x_1 + 3 x_2 + 7 x_3 + x_4 + 4 x_5 + 8 x_6 + 3 x_7 + 4 x_8 <= 30
constr(3):  6 x_0 + 3 x_1 + 9 x_2 + 5 x_3 + 9 x_4 + 6 x_5 + 9 x_6 + 9 x_7 + 6 x_8 <= 40
constr(4):  10 x_0 + 8 x_1 + 8 x_2 + 7 x_3 + 10 x_4 + 10 x_5 + 9 x_6 + 9 x_7 + 3 x_8 <= 20
constr(5):  10 x_0 + 10 x_1 + 10 x_2 + 9 x_3 + 10 x_4 + x_5 + 8 x_6 + 3 x_7 + 10 x_8 <= 90
constr(6):  10 x_0 

In [ ]:
fres = open(f"resposta4.txt", "r")
print(fres.read())
fres.close()

--------------------------------------------------------------------------------------------------------------------------------------------
Iteração 1
Resultado da Relaxação Linear: 13.571
Variáveis: 
x_0 = 0.00
x_1 = 0.00
x_2 = 0.71
x_3 = 0.00
x_4 = 0.00
x_5 = 0.00
x_6 = 0.00
x_7 = 0.00
x_8 = 0.71
Variável escolhida: x_2 = 0.71

Restrições:

\Problem name: 

Minimize
OBJROW: -9 x_0 -7 x_1 -10 x_2 -7 x_3 -9 x_4 -6 x_5 -8 x_6 -4 x_7 -9 x_8
Subject To
constr(0):  4 x_0 + 9 x_1 + 4 x_2 + x_3 + 9 x_4 + 6 x_5 + 3 x_6 + 6 x_7 + x_8 <= 40
constr(1):  3 x_0 + 7 x_1 + 8 x_2 + 7 x_3 + 6 x_4 + 3 x_5 + 5 x_6 + 9 x_7 + 4 x_8 <= 80
constr(2):  9 x_0 + 3 x_1 + 6 x_2 + 5 x_3 + 7 x_4 + x_5 + x_6 + 3 x_7 + 9 x_8 <= 40
constr(3):  5 x_0 + 9 x_1 + 6 x_2 + 5 x_3 + 9 x_4 + 7 x_5 + 8 x_6 + 7 x_7 + 8 x_8 <= 10
constr(4):  7 x_0 + 7 x_1 + 4 x_2 + x_3 + 3 x_4 + 4 x_5 + 8 x_6 + x_7 + 9 x_8 <= 10
constr(5):  x_0 + 6 x_1 + 6 x_2 + x_3 + 6 x_4 + 7 x_5 + 3 x_6 + 8 x_7 + 7 x_8 <= 10
constr(6):  6 x_0 + 6 x_1 + 8 x_2